In [1]:
import tensorflow_decision_forests as tfdf

import tensorflow as tf

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import math

import dtreeviz

from matplotlib import pyplot as plt
from IPython import display

ModuleNotFoundError: No module named 'tensorflow_decision_forests'

In [ ]:
# avoid "Arial font not found warnings"
import logging
logging.getLogger('matplotlib.font_manager').setLevel(level=logging.CRITICAL)

display.set_matplotlib_formats('retina') # generate hires plots

np.random.seed(1234)  # reproducible plots/data for explanatory reasons

In [ ]:
# Split the dataset into a training and a testing dataset.

def split_dataset(dataset, test_ratio=0.30):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]



In [ ]:
df = pd.read_csv('heart.csv', sep=",")

In [ ]:
train_ds_pd, test_ds_pd = split_dataset(df)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))

In [ ]:
df

In [ ]:
target = 'target'

classes = list(df[target].unique())

df[target] = df[target].map(classes.index)

In [ ]:
# Split into training and test sets
train_ds_pd, test_ds_pd = split_dataset(df)
print(f"{len(train_ds_pd)} examples in training, {len(test_ds_pd)} examples for testing.")

# Convert to tensorflow data sets
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=target)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=target)

In [ ]:
# Specify the model.
model_1 = tfdf.keras.RandomForestModel()

# Train the model.
model_1.fit(train_ds, validation_data=test_ds)

In [ ]:
model_1.compile(metrics=["accuracy"])
evaluation = model_1.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

In [ ]:
model_1.save("/tmp/my_saved_model")

In [ ]:
model_1.summary()

In [ ]:
# The input features
model_1.make_inspector().features()

In [ ]:
# The feature importances
model_1.make_inspector().variable_importances()

In [ ]:
# Tell dtreeviz about training data and model
features = [f.name for f in model_1.make_inspector().features()]
viz_cmodel = dtreeviz.model(model_1,
                           tree_index=3,
                           X_train=train_ds_pd[features],
                           y_train=train_ds_pd[target],
                           feature_names=features,
                           target_name=target,
                           class_names=classes)

In [ ]:
viz_cmodel.view(scale=1.2)

In [ ]:
print(features)

In [ ]:
model_1.save("my_model")

In [ ]:
df.head(1)

df[~'infectionProb']

In [ ]:
test = tfdf.keras.pd_dataframe_to_tf_dataset(df.loc[:, df.columns!='target'].head(1))
t1 = df['target'].head(1)

In [ ]:
tes = model_1.predict(test, verbose=0)

In [ ]:
k = []
i = 0
cnt = 0
for t in tes:
    if(t[0]>0.50):
        k.append(1)
        #print("1 - " + str(t1[i]))
        if(t1[i] == 1):
            cnt += 1
    else:
        k.append(0)
        #print("0 - " + str(t1[i]))
        if(t1[i] == 0):
            cnt += 1
    i+=1
print(cnt/len(t1))

In [ ]:
print(df.head(1))

In [ ]:
t2_dic = {'age':[103], 'cp': [0], 'fbs':[1], 'thalach ':[170]}
t2_df = pd.DataFrame.from_dict(t2_dic)

In [ ]:
d2_tfdf = tfdf.keras.pd_dataframe_to_tf_dataset(t2_df)

In [ ]:
model_1.predict(d2_tfdf, verbose=0)